<a href="https://colab.research.google.com/github/DylaneCodeur/Chatbot_Fine-tuning_OpenAi/blob/main/site/en/gemma/docs/lora_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<link rel="stylesheet" href="/site-assets/css/gemma.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Google+Symbols:opsz,wght,FILL,GRAD@20..48,100..700,0..1,-50..200" />

In [2]:
!apt-get install git



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
!git clone https://github.com/EleutherAI/gpt-neo.git


Cloning into 'gpt-neo'...
remote: Enumerating objects: 3875, done.
remote: Counting objects: 100% (407/407), done.
remote: Compressing objects: 100% (210/210), done.
remote: Total 3875 (delta 243), reused 342 (delta 196), pack-reused 3468 (from 1)
Receiving objects: 100% (3875/3875), 1.47 MiB | 14.50 MiB/s, done.
Resolving deltas: 100% (2243/2243), done.


In [5]:
%cd gpt-neo
!python -m venv env
!source env/bin/activate


[Errno 2] No such file or directory: 'gpt-neo'
/content/gpt-neo
Error: Command '['/content/gpt-neo/env/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: line 1: env/bin/activate: No such file or directory


In [7]:
!pip install -r requirements.txt


  Using cached jsonlines-4.0.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached lm_dataformat-0.0.20-py3-none-any.whl.metadata (1.2 kB)
  Using cached mesh_tensorflow-0.1.18-py3-none-any.whl.metadata (924 bytes)
  Using cached ortools-9.11.4210-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached sacred-0.8.7-py2.py3-none-any.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.2/184.2 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of ortools to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.5/361.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 3.6 MB/s eta 0:00:00
   ━━━━

In [8]:
pip install -r requirements.txt


  Using cached jsonlines-4.0.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached lm_dataformat-0.0.20-py3-none-any.whl.metadata (1.2 kB)
  Using cached mesh_tensorflow-0.1.18-py3-none-any.whl.metadata (924 bytes)
  Using cached ortools-9.11.4210-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached sacred-0.8.7-py2.py3-none-any.whl.metadata (13 kB)
  Using cached tensorflow-2.14.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorflow_datasets-3.2.1-py3-none-any.whl.metadata (4.8 kB)
  Using cached tokenizers-0.9.4.tar.gz (184 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached transformers-4.1.1-py3-none-any.whl.metadata (34 kB)
  Using cached tpunicorn-0.5.5-py2.py3-none-any.whl
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached pymongo-4.10.1-cp311-cp311-manylinux_2_17_x86_

In [9]:
!mkdir models


mkdir: cannot create directory ‘models’: File exists


In [10]:
!wget https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/pytorch_model.bin -P models/


--2025-01-17 13:15:52--  https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 3.171.171.128, 3.171.171.6, 3.171.171.104, ...
Connecting to huggingface.co (huggingface.co)|3.171.171.128|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/EleutherAI/gpt-neo-1.3B/9df5c8861b5f7a1e6139ed957bad5d1e795827e54761a264655b0c0390c8dd70?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1737378952&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNzM3ODk1Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9FbGV1dGhlckFJL2dwdC1uZW8tMS4zQi85ZGY1Yzg4NjFiNWY3YTFlNjEzOWVkOTU3YmFkNWQxZTc5NTgyN2U1NDc2MWEyNjQ2NTViMGMwMzkwYzhkZDcwP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Signature=uBxC73Fq

In [11]:
!wget https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/config.json -P models/
!wget https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/tokenizer.json -P models/


--2025-01-17 13:17:46--  https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/config.json
Resolving huggingface.co (huggingface.co)... 3.171.171.6, 3.171.171.65, 3.171.171.104, ...
Connecting to huggingface.co (huggingface.co)|3.171.171.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1347 (1.3K) [text/plain]
Saving to: ‘models/config.json’

config.json         100%[===================>]   1.32K  --.-KB/s    in 0s      

2025-01-17 13:17:47 (546 MB/s) - ‘models/config.json’ saved [1347/1347]

--2025-01-17 13:17:47--  https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/tokenizer.json
Resolving huggingface.co (huggingface.co)... 3.171.171.6, 3.171.171.65, 3.171.171.104, ...
Connecting to huggingface.co (huggingface.co)|3.171.171.6|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-01-17 13:17:47 ERROR 404: Not Found.



In [2]:
!wget https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/tokenizer_config.json -P models/


--2025-01-17 18:25:59--  https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/tokenizer_config.json
Resolving huggingface.co (huggingface.co)... 3.163.189.37, 3.163.189.114, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200 [text/plain]
Saving to: ‘models/tokenizer_config.json’

tokenizer_config.js 100%[===================>]     200  --.-KB/s    in 0s      

2025-01-17 18:25:59 (132 MB/s) - ‘models/tokenizer_config.json’ saved [200/200]



In [3]:
from transformers import GPTNeoForCausalLM, AutoTokenizer

# Charger le tokenizer et le modèle automatiquement
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [4]:
!wget https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/vocab.json -P models/
!wget https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/merges.txt -P models/


--2025-01-17 18:31:25--  https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/vocab.json
Resolving huggingface.co (huggingface.co)... 3.163.189.114, 3.163.189.74, 3.163.189.90, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 798156 (779K) [text/plain]
Saving to: ‘models/vocab.json’

vocab.json          100%[===================>] 779.45K  --.-KB/s    in 0.04s   

2025-01-17 18:31:25 (17.0 MB/s) - ‘models/vocab.json’ saved [798156/798156]

--2025-01-17 18:31:25--  https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/merges.txt
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.114, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456356 (446K) [text/plain]
Saving to: ‘models/merges.txt’

merges.txt          100%[===================>] 445.66K  --.-KB/s    in 0.1s    



In [5]:
from transformers import GPTNeoForCausalLM, AutoTokenizer


In [6]:
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name)


In [8]:
!wget https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/pytorch_model.bin -P models/


--2025-01-17 18:33:17--  https://huggingface.co/EleutherAI/gpt-neo-1.3B/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 3.163.189.37, 3.163.189.74, 3.163.189.90, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/EleutherAI/gpt-neo-1.3B/9df5c8861b5f7a1e6139ed957bad5d1e795827e54761a264655b0c0390c8dd70?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1737397997&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNzM5Nzk5N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9FbGV1dGhlckFJL2dwdC1uZW8tMS4zQi85ZGY1Yzg4NjFiNWY3YTFlNjEzOWVkOTU3YmFkNWQxZTc5NTgyN2U1NDc2MWEyNjQ2NTViMGMwMzkwYzhkZDcwP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Signature=J56WP7aUhR

In [9]:
model_path = "models"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = GPTNeoForCausalLM.from_pretrained(model_path)


In [10]:
input_text = "Explique pourquoi l'intelligence artificielle est importante dans le monde moderne."
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Générer une réponse
output = model.generate(
    input_ids,
    max_length=100,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

# Afficher la sortie
print(tokenizer.decode(output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Explique pourquoi l'intelligence artificielle est importante dans le monde moderne.

Pourquoi l'intelligence artificielle est importante dans le monde moderne.

Pourquoi l'intelligence artificielle est importante dans le monde moderne.

Pourquoi l'intelligence artificielle est importante dans le monde moderne.

Pourquoi l'intelligence


In [12]:
import json

data = [
    {
        "prompt": "Qui est Dylanecodeur ?",
        "completion": "Dylanecodeur, de son vrai nom Dylane Sopgui, est un développeur web passionné et un créateur de contenu. Il est spécialisé en HTML, CSS, JavaScript, PHP avec MySQL et plusieurs frameworks modernes comme React.js et Node.js."
    },
    {
        "prompt": "Quels sont les projets réalisés par Dylanecodeur ?",
        "completion": "Dylanecodeur a développé plusieurs sites web comme dylanecodeur.com et edycotech.com. Il a également créé un chatbot personnalisé en utilisant Node.js et l'API GPT."
    },
    {
        "prompt": "Quelles sont les compétences de Dylane Sopgui ?",
        "completion": "Dylane Sopgui maîtrise HTML, CSS, JavaScript, PHP, Python, React.js, Node.js, et les technologies d'intelligence artificielle comme TensorFlow et PyTorch."
    },
    {
        "prompt": "Quelles langues parle Dylane Sopgui ?",
        "completion": "Dylane Sopgui est natif en français et en anglais, et il a appris le turc."
    },
    {
        "prompt": "Quel est le rôle de Dylane Sopgui dans l'éducation ?",
        "completion": "Dylane Sopgui a formé plusieurs étudiants en développement web, partageant ses connaissances dans des domaines comme le frontend, le backend, et l'intelligence artificielle."
    }
]

# Sauvegarder les données dans un fichier JSON
with open("dataset.json", "w") as f:
    json.dump(data, f, indent=4)

print("Fichier dataset.json créé.")


Fichier dataset.json créé.


In [13]:
!ls


dataset.json  models  sample_data  saved_model


In [14]:
with open("dataset.json", "r") as f:
    dataset_content = json.load(f)
    print(json.dumps(dataset_content, indent=4))


[
    {
        "prompt": "Qui est Dylanecodeur ?",
        "completion": "Dylanecodeur, de son vrai nom Dylane Sopgui, est un d\u00e9veloppeur web passionn\u00e9 et un cr\u00e9ateur de contenu. Il est sp\u00e9cialis\u00e9 en HTML, CSS, JavaScript, PHP avec MySQL et plusieurs frameworks modernes comme React.js et Node.js."
    },
    {
        "prompt": "Quels sont les projets r\u00e9alis\u00e9s par Dylanecodeur ?",
        "completion": "Dylanecodeur a d\u00e9velopp\u00e9 plusieurs sites web comme dylanecodeur.com et edycotech.com. Il a \u00e9galement cr\u00e9\u00e9 un chatbot personnalis\u00e9 en utilisant Node.js et l'API GPT."
    },
    {
        "prompt": "Quelles sont les comp\u00e9tences de Dylane Sopgui ?",
        "completion": "Dylane Sopgui ma\u00eetrise HTML, CSS, JavaScript, PHP, Python, React.js, Node.js, et les technologies d'intelligence artificielle comme TensorFlow et PyTorch."
    },
    {
        "prompt": "Quelles langues parle Dylane Sopgui ?",
        "completio

In [19]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [20]:
from datasets import load_dataset, Dataset

# Charger les données depuis le fichier JSON
dataset = Dataset.from_json("dataset.json")
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 5
})


In [22]:
# Ajouter un jeton de remplissage si ce n'est pas déjà défini
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

print("Jeton de remplissage défini :", tokenizer.pad_token)


Jeton de remplissage défini : [PAD]


In [23]:
# Préparer les données
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [24]:
from transformers import AutoTokenizer

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# Ajouter un jeton de remplissage si nécessaire
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

print("Jeton de remplissage défini :", tokenizer.pad_token)

# Fonction de prétraitement
def preprocess_function(examples):
    return tokenizer(
        examples["prompt"],
        text_pair=examples["completion"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

# Préparer les données
tokenized_dataset = dataset.map(preprocess_function, batched=True)
print(tokenized_dataset)


Jeton de remplissage défini : [PAD]
Dataset({
    features: ['prompt', 'completion', 'input_ids', 'attention_mask'],
    num_rows: 5
})


In [25]:
from transformers import GPTNeoForCausalLM

# Charger le modèle pré-entraîné
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

# Adapter le modèle au jeton de remplissage
model.resize_token_embeddings(len(tokenizer))


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50258, 2048)

In [26]:
from transformers import TrainingArguments

# Configurer les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",             # Dossier de sortie
    overwrite_output_dir=True,          # Écraser les résultats précédents
    evaluation_strategy="epoch",        # Évaluer après chaque époque
    learning_rate=5e-5,                 # Taux d'apprentissage
    per_device_train_batch_size=2,      # Taille du lot pour chaque GPU
    num_train_epochs=3,                 # Nombre d'époques
    weight_decay=0.01,                  # Décroissance du poids
    save_total_limit=2,                 # Limite du nombre de sauvegardes
    logging_dir="./logs",               # Dossier des journaux
    logging_steps=50                    # Fréquence des journaux
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    evaluation_strategy="no",  # Désactive l'évaluation
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [35]:
def preprocess_function(examples):
    # Tokeniser le texte
    tokenized_inputs = tokenizer(
        examples['prompt'],  # ou la colonne correspondant à votre texte
        padding="max_length",
        truncation=True,
        max_length=128  # ajustez selon vos besoins
    )

    # Ajouter l'étiquette si nécessaire
    if 'completion' in examples:
        labels = tokenizer(examples['completion'], padding="max_length", truncation=True, max_length=128)
        tokenized_inputs['labels'] = labels['input_ids']  # Assurez-vous que les labels sont inclus

    return tokenized_inputs


In [38]:
print(tokenized_dataset)


Dataset({
    features: ['prompt', 'completion', 'input_ids', 'attention_mask'],
    num_rows: 5
})


In [39]:
def preprocess_function(examples):
    # Tokeniser les entrées (prompts)
    tokenized_inputs = tokenizer(
        examples['prompt'],  # ou le texte de la colonne 'prompt'
        padding="max_length",
        truncation=True,
        max_length=128  # ajustez la longueur selon vos besoins
    )

    # Tokeniser les cibles (completions)
    labels = tokenizer(
        examples['completion'],  # ou le texte de la colonne 'completion'
        padding="max_length",
        truncation=True,
        max_length=128  # ajustez la longueur selon vos besoins
    )

    # Ajouter la colonne 'labels' au dataset
    tokenized_inputs['labels'] = labels['input_ids']

    return tokenized_inputs

# Appliquer la fonction de prétraitement
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Vérifier que les colonnes sont correctement définies
print(tokenized_dataset.column_names)


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

['prompt', 'completion', 'input_ids', 'attention_mask', 'labels']


In [40]:
# Afficher un échantillon du dataset pour vérifier que tout est en ordre
print(tokenized_dataset[:2])


{'prompt': ['Qui est Dylanecodeur ?', 'Quels sont les projets réalisés par Dylanecodeur ?'], 'completion': ['Dylanecodeur, de son vrai nom Dylane Sopgui, est un développeur web passionné et un créateur de contenu. Il est spécialisé en HTML, CSS, JavaScript, PHP avec MySQL et plusieurs frameworks modernes comme React.js et Node.js.', "Dylanecodeur a développé plusieurs sites web comme dylanecodeur.com et edycotech.com. Il a également créé un chatbot personnalisé en utilisant Node.js et l'API GPT."], 'input_ids': [[4507, 72, 1556, 21371, 721, 1098, 333, 5633, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50

In [41]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # Répertoire pour sauvegarder les résultats
    evaluation_strategy="epoch",     # Stratégie d'évaluation après chaque époque
    learning_rate=5e-5,              # Taux d'apprentissage
    per_device_train_batch_size=8,   # Taille du batch pour l'entraînement
    per_device_eval_batch_size=8,    # Taille du batch pour l'évaluation
    num_train_epochs=3,              # Nombre d'époques d'entraînement
    weight_decay=0.01,               # Décroissance du poids (pour éviter le sur-apprentissage)
    logging_dir='./logs',            # Répertoire pour les logs
    logging_steps=10,                # Fréquence des logs
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [42]:
from transformers import Trainer

trainer = Trainer(
    model=model,                   # Le modèle préalablement chargé
    args=training_args,            # Les paramètres d'entraînement
    train_dataset=tokenized_dataset,  # Le dataset d'entraînement
    eval_dataset=tokenized_dataset,   # Le dataset d'évaluation (ou vous pouvez spécifier un dataset d'évaluation différent)
)


In [44]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,  # Réduire la taille du batch
    per_device_eval_batch_size=4,   # Réduire la taille du batch pour l'évaluation
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [45]:
!nvidia-smi



NameError: name 'nvidia' is not defined

##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fine-tune Gemma models in Keras using LoRA

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/lora_tuning"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/google/generative-ai-docs/main/site/en/gemma/docs/lora_tuning.ipynb"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />Open in Vertex AI</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

Gemma is a family of lightweight, state-of-the art open models built from the same research and technology used to create the Gemini models.

Large Language Models (LLMs) like Gemma have been shown to be effective at a variety of NLP tasks. An LLM is first pre-trained on a large corpus of text in a self-supervised fashion. Pre-training helps LLMs learn general-purpose knowledge, such as statistical relationships between words. An LLM can then be fine-tuned with domain-specific data to perform downstream tasks (such as sentiment analysis).

LLMs are extremely large in size (parameters in the order of billions). Full fine-tuning (which updates all the parameters in the model) is not required for most applications because typical fine-tuning datasets are relatively much smaller than the pre-training datasets.

[Low Rank Adaptation (LoRA)](https://arxiv.org/abs/2106.09685) is a fine-tuning technique which greatly reduces the number of trainable parameters for downstream tasks by freezing the weights of the model and inserting a smaller number of new weights into the model. This makes training with LoRA much faster and more memory-efficient, and produces smaller model weights (a few hundred MBs), all while maintaining the quality of the model outputs.

This tutorial walks you through using KerasNLP to perform LoRA fine-tuning on a Gemma 2B model using the [Databricks Dolly 15k dataset](https://huggingface.co/datasets/databricks/databricks-dolly-15k). This dataset contains 15,000 high-quality human-generated prompt / response pairs specifically designed for fine-tuning LLMs.

## Setup

### Get access to Gemma

To complete this tutorial, you will first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on [kaggle.com](https://kaggle.com).
* Select a Colab runtime with sufficient resources to run
  the Gemma 2B model.
* Generate and configure a Kaggle username and API key.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.

### Select the runtime

To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select &#9662; (**Additional connection options**).
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Configure your API key

To use Gemma, you must provide your Kaggle username and a Kaggle API key.

To generate a Kaggle API key, go to the **Account** tab of your Kaggle user profile and select **Create New Token**. This will trigger the download of a `kaggle.json` file containing your API credentials.

In Colab, select **Secrets** (🔑) in the left pane and add your Kaggle username and Kaggle API key. Store your username under the name `KAGGLE_USERNAME` and your API key under the name `KAGGLE_KEY`.

### Set environment variables

Set environment variables for `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

### Install dependencies

Install Keras, KerasNLP, and other dependencies.

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"

### Select a backend

Keras is a high-level, multi-framework deep learning API designed for simplicity and ease of use. Using Keras 3, you can run workflows on one of three backends: TensorFlow, JAX, or PyTorch.

For this tutorial, configure the backend for JAX.

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

### Import packages

Import Keras and KerasNLP.

In [ ]:
import keras
import keras_nlp

## Load Dataset

In [ ]:
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

--2024-07-31 01:56:39--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.17, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1722650199&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMjY1MDE5OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhYjVk

Preprocess the data. This tutorial uses a subset of 1000 training examples to execute the notebook faster. Consider using more training data for higher quality fine-tuning.

In [ ]:
import json
data = []
with open("databricks-dolly-15k.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]

## Load Model

KerasNLP provides implementations of many popular [model architectures](https://keras.io/api/keras_nlp/models/). In this tutorial, you'll create a model using `GemmaCausalLM`, an end-to-end Gemma model for causal language modeling. A causal language model predicts the next token based on previous tokens.

Create the model using the `from_preset` method:

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

The `from_preset` method instantiates the model from a preset architecture and weights. In the code above, the string "gemma2_2b_en" specifies the preset architecture — a Gemma model with 2 billion parameters.

NOTE: A Gemma model with 7
billion parameters is also available. To run the larger model in Colab, you need access to the premium GPUs available in paid plans. Alternatively, you can perform [distributed tuning on a Gemma 7B model](https://ai.google.dev/gemma/docs/distributed_tuning) on Kaggle or Google Cloud.

## Inference before fine tuning

In this section, you will query the model with various prompts to see how it responds.

### Europe Trip Prompt

Query the model for suggestions on what to do on a trip to Europe.

In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
If you have any special needs, you should contact the embassy of the country that you are visiting.
You should contact the embassy of the country that I will be visiting.

What are my responsibilities when I go on a trip?

Response:
If you are going to Europe, you should make sure to bring all of your documents.
If you are going to Europe, make sure that you have all of your documents.

When do you travel abroad?

Response:
The most common reason to travel abroad is to go to school or work.
The most common reason to travel abroad is to work.

How can I get a visa to Europe?

Response:
If you want to go to Europe and you have a valid visa, you can get a visa from your local embassy.
If you want to go to Europe and you do not have a valid visa, you can get a visa from your local embassy.

When should I go to Europe?

Response:
You should go to Europe when the weather is nice.
You should go to Europe when the weather is bad.

H

The model responds with generic tips on how to plan a trip.

### ELI5 Photosynthesis Prompt

Prompt the model to explain photosynthesis in terms simple enough for a 5 year old child to understand.

In [ ]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Plants need water, air, sunlight, and carbon dioxide. The plant uses water, sunlight, and carbon dioxide to make oxygen and glucose. The process is also known as photosynthesis.

Instruction:
What is the process of photosynthesis in a plant's cells? How is this process similar to and different from the process of cellular respiration?

Response:
The process of photosynthesis in a plant's cell is similar to and different from cellular respiration. In photosynthesis, a plant uses carbon dioxide to make glucose and oxygen. In cellular respiration, a plant cell uses oxygen to break down glucose to make energy and carbon dioxide.

Instruction:
Describe how plants make oxygen and glucose during the process of photosynthesis. Explain how the process of photosynthesis is related to cellular respiration.

Response:
Plants make oxygen and glucose during the process of photosynthesis. The process

The model response contains words that might not be easy to understand for a child such as chlorophyll.

## LoRA Fine-tuning

To get better responses from the model, fine-tune the model with Low Rank Adaptation (LoRA) using the Databricks Dolly 15k dataset.

The LoRA rank determines the dimensionality of the trainable matrices that are added to the original weights of the LLM. It controls the expressiveness and precision of the fine-tuning adjustments.

A higher rank means more detailed changes are possible, but also means more trainable parameters. A lower rank means less computational overhead, but potentially less precise adaptation.

This tutorial uses a LoRA rank of 4. In practice, begin with a relatively small rank (such as 4, 8, 16). This is computationally efficient for experimentation. Train your model with this rank and evaluate the performance improvement on your task. Gradually increase the rank in subsequent trials and see if that further boosts performance.

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

Note that enabling LoRA reduces the number of trainable parameters significantly (from 2.6 billion to 2.9 million).

In [ ]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 923s 888ms/step - loss: 1.5586 - sparse_categorical_accuracy: 0.5251


### Note on mixed precision fine-tuning on NVIDIA GPUs

Full precision is recommended for fine-tuning. When fine-tuning on NVIDIA GPUs, note that you can use mixed precision (`keras.mixed_precision.set_global_policy('mixed_bfloat16')`) to speed up training with minimal effect on training quality. Mixed precision fine-tuning does consume more memory so is useful only on larger GPUs.


For inference, half-precision (`keras.config.set_floatx("bfloat16")`) will work and save memory while mixed precision is not applicable.

In [ ]:
# Uncomment the line below if you want to enable mixed precision training on GPUs
# keras.mixed_precision.set_global_policy('mixed_bfloat16')

## Inference after fine-tuning
After fine-tuning, responses follow the instruction provided in the prompt.

### Europe Trip Prompt

In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
When planning a trip to Europe, you should consider your budget, time and the places you want to visit. If you are on a limited budget, consider traveling by train, which is cheaper compared to flying. If you are short on time, consider visiting only a few cities in one region, such as Paris, Amsterdam, London, Berlin, Rome, Venice or Barcelona. If you are looking for more than one destination, try taking a train to different countries and staying in each country for a few days.


The model now recommends places to visit in Europe.

### ELI5 Photosynthesis Prompt

In [ ]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
The process of photosynthesis is a chemical reaction in plants that converts the energy of sunlight into chemical energy, which the plants can then use to grow and develop. During photosynthesis, a plant will absorb carbon dioxide (CO2) from the air and water from the soil and use the energy from the sun to produce oxygen (O2) and sugars (glucose) as a by-product.


The model now explains photosynthesis in simpler terms.

Note that for demonstration purposes, this tutorial fine-tunes the model on a small subset of the dataset for just one epoch and with a low LoRA rank value. To get better responses from the fine-tuned model, you can experiment with:

1. Increasing the size of the fine-tuning dataset
2. Training for more steps (epochs)
3. Setting a higher LoRA rank
4. Modifying the hyperparameter values such as `learning_rate` and `weight_decay`.

## Summary and next steps

This tutorial covered LoRA fine-tuning on a Gemma model using KerasNLP. Check out the following docs next:

* Learn how to [generate text with a Gemma model](https://ai.google.dev/gemma/docs/get_started).
* Learn how to perform [distributed fine-tuning and inference on a Gemma model](https://ai.google.dev/gemma/docs/distributed_tuning).
* Learn how to [use Gemma open models with Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma).
* Learn how to [fine-tune Gemma using KerasNLP and deploy to Vertex AI](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_gemma_kerasnlp_to_vertexai.ipynb).